In [1]:
!pip install accelerate datasets evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00


In [17]:
model_name = "sbunlp/fabert"

In [6]:
from datasets import load_dataset, DatasetDict, Dataset

dsp = load_dataset("Khedesh/DeepSentiPers")
v_ = dsp.pop("validation")
dsp['test'] = v_
dsptr = dsp['train'].train_test_split(test_size = 0.1,seed = 42)
dsp['train'] = dsptr['train']
dsp['valid'] = dsptr['test']


In [7]:
# convert from 5-label to ternary (-1,0,1)
lmap = {0:1,-1:0,-2:0,1:2,2:2}
dsp_ternary = DatasetDict({
    "train": Dataset.from_dict({
        "text": dsp["train"]["text"],
        "label": [lmap[label] for label in dsp["train"]["label"]],
    }),
    "test": Dataset.from_dict({
        "text": dsp["test"]["text"],
        "label": [lmap[label] for label in dsp["test"]["label"]],
    }),
    "valid": Dataset.from_dict({
        "text": dsp["valid"]["text"],
        "label": [lmap[label] for label in dsp["valid"]["label"]],
    })
})

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/552k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [26]:
def preprocess_function(examples):
    tokenized_text = tokenizer(examples['text'], truncation=True, padding="max_length", max_length=512)
    return tokenized_text

In [27]:
tokenized_dsp = dsp_ternary.map(preprocess_function, batched = True)

Map:   0%|          | 0/6320 [00:00<?, ? examples/s]

Map:   0%|          | 0/1854 [00:00<?, ? examples/s]

Map:   0%|          | 0/703 [00:00<?, ? examples/s]

In [13]:
import numpy as np
import evaluate
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_result = accuracy.compute(predictions=predictions, references=labels)['accuracy']
    f1_result = f1.compute(predictions=predictions, references=labels,average='macro')['f1']
    return {"accuracy":accuracy_result,"f1":f1_result}


In [15]:
id2label = {
    0:"Negative",
    1:"Neutral",
    2:"Positive",
}
label2id = {
    "Negative":0,
    "Neutral":1,
    "Positive":2,
}

In [29]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [57]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label),id2label=id2label,label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sbunlp/fabert and are newly initialized: ['bert.pooler.dense.bias', 'classifier.bias', 'bert.pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
training_args = TrainingArguments(
    output_dir = "/tmp/dsp",
    learning_rate= 5e-5,
    per_device_train_batch_size= 16,
    per_device_eval_batch_size= 16,
    num_train_epochs= 3,
    weight_decay= 0,
    warmup_ratio = 0,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dsp["train"],
    eval_dataset=tokenized_dsp["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.592956,0.776671,0.722097
2,0.661100,0.569104,0.800853,0.759889
3,0.356500,0.582801,0.807966,0.763582


TrainOutput(global_step=1185, training_loss=0.472408964261727, metrics={'train_runtime': 1765.0395, 'train_samples_per_second': 10.742, 'train_steps_per_second': 0.671, 'total_flos': 4988630400122880.0, 'train_loss': 0.472408964261727, 'epoch': 3.0})

In [59]:
validation_results = trainer.evaluate()

In [60]:
print(validation_results)

{'eval_loss': 0.5828011631965637, 'eval_accuracy': 0.8079658605974396, 'eval_f1': 0.7635819614521666, 'eval_runtime': 22.2554, 'eval_samples_per_second': 31.588, 'eval_steps_per_second': 1.977, 'epoch': 3.0}


In [61]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dsp["train"],
    eval_dataset=tokenized_dsp["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
test_results = trainer.evaluate()

In [62]:
print(test_results)

{'eval_loss': 0.5144689083099365, 'eval_accuracy': 0.8214670981661273, 'eval_f1': 0.7871936184405417, 'eval_runtime': 59.3384, 'eval_samples_per_second': 31.245, 'eval_steps_per_second': 1.955}


# Pipeline

In [63]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model = model,tokenizer = tokenizer,device=0)

In [64]:
text = "از این خرید بسیار راضی هستم."
classifier(text)

[{'label': 'Positive', 'score': 0.9920294284820557}]

In [65]:
text = "پیشنهاد میکنم این محصول رو نگیرید"
classifier(text)

[{'label': 'Negative', 'score': 0.611400842666626}]